## 8. When Models Meet Data

#### 8.1.1 Data as Vectors

The data represent on table with N as number of examples (row) and D (dimension) as number of features (columns)

We can manipulate the data find potentially better representations by two ways:
1. finding lower-dimensional approximations of the original feature vector -> Principal Component Analysis (PCA), closely related to concepts of __eigenvalue__ and **singular value decomposition**
2. using nonlinear higher-dimensional combinations of the original feature vector -> explicit feature map(?), the main motivation for higher-dimensional representations is that we can construct new features which may take the learning problem easier(?)

##### (Math Check) Eigenvalue

##### (Math Check) 2.7.1 Matrix Representation of Linear Mapping
vectors are objects that can be added together and multiplied by a scalar, and the resulting object is still a vector
notation:
1. B = (b1,...,bn) -> ordered basis
2. B = {b1,...,bn} -> unordered basis
3. B = [b1,...,bn] -> matrix whose columns are vectors

definition 2.18 (coordinates) -> this definitions is changing the existing vector on standard basis (e1, e2) to basis vector (I think this customizeable(?)) 

Example of coordinates changes on vector using basis vector (b1, b2) -> b1 = [1, -1]^T, b2 = [1, 1]^T

we have vector \
x = 2*e1 + 3*e2 

standard basis \
e1 = [1, 0]^T \
e2 = [0, 2]^T

vector x can be written as: \
x = 2 * [1, 0]^T + 3 * [0, 1]^T = [2, 3]^T

instead of using standard basis, we want to **represent the same vector** x using a different basis (b1, b2) \
b1 = [1, -1]^T \
b2 = [1, 1]^T 

x as linear combination of b1 and b2 \
x = c1*b1 + c2*b2

finding new coordinate c1, c2 \
x = c1*[1, -1]^T + c2*[1, 1]^T \
x = [[1, 1], [-1, 1]] * [c1, c2]^T = [2, 3]^T \
[c1, c2]^T = [[1, 1], [-1, 1]]^-1 * [2, 3]^T

[[1, 1], [-1, 1]]^-1: \
determinant = (1*1) - ((-1)*1) = 1 + 1 = 2

[[1, 1], [-1, 1]]^-1 = 0.5 * ( [[1, -1], [1, 1]] ) 

[c1, c2]^T = ( 0.5 * ( [[1, -1], [1, 1]] ) ) * [2, 3]^T

[c1, c2]^T = [-0.5, 2.5]^T

In [63]:
## Manual code

# original vector x
x = [[2], [3]]

# basis b1, b2
B = [
    [1, 1],
    [-1, 1]
]

def determinant_2x2(matrix):
    return (matrix[0][0] * matrix[1][1]) - (matrix[0][1] * matrix[1][0]) 

def inverse_2x2(matrix):
    det = determinant_2x2(matrix)

    if det == 0:
        raise ValueError("Matrix is singular, cannto find inverse.")

    inv_det = 1 / det
    inverse = [
        [matrix[1][1] * inv_det, -1 * matrix[0][1] * inv_det],
        [-1 * matrix[1][0] * inv_det, matrix [0][0] * inv_det]
    ]

    return inverse

B_inv= inverse_2x2(B)

def matrix_mult(B_inv, x):
    ## multiply two matrices A (m x n) and B (n x p)
    m = len(B_inv)
    n = len(B_inv[0])
    p = len(x[0]) 

    result = [[0] for _ in range(len(x))]

    for i from range(m):
        for j from range(p):
            for k from range(n)


In [121]:
B_inv[0][0]

0.5

In [123]:
B_inv[0][1]

-0.5

In [125]:
B_inv

[[0.5, -0.5], [0.5, 0.5]]